<a href="https://colab.research.google.com/github/stun100/Audio-to-text/blob/main/audio_to_text_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Requirements

In [ ]:
!pip install faster-whisper
!pip install pydub
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0

# 2. Imports

In [ ]:
from faster_whisper import WhisperModel
import time
from pydub import AudioSegment
import math
import os
import openai
import requests

openai.api_key = ""

def split_audio(input_path, output_path, segment_length_min):

    filename, audio_format = filename_and_format_from_path(input_path)

    audio_file = AudioSegment.from_file(input_path, format=audio_format)

    # set the segment length to 15 minutes (in milliseconds)
    segment_length_millisec = segment_length_min * 60 * 1000

    num_segments = math.ceil(len(audio_file) / segment_length_millisec)

    for i in range(num_segments):
        start = i * segment_length_millisec
        end = min((i+1) * segment_length_millisec, len(audio_file))
        segment = audio_file[start:end]
        if (output_path == ""):
            segment.export(f"{filename}_{i+1}.mp3", format="mp3", codec="libmp3lame")
        else:
            segment.export(f"{output_path}/{filename}_{i+1}.mp3", format="mp3", codec="libmp3lame")


def filename_and_format_from_path(path):
    splitted_path = path.split("/")
    return splitted_path[-1].split(".")

# 3. Code

## 3.1 Splitting audio file in multiple segments

In [ ]:
numero_lezione = 8
numero_di_audio = 1
input_path = f"/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_original"
output_path = f"/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_segments"

In [ ]:
#argomento = "Drammaturgia"

for i in range(numero_di_audio):
  split_audio(input_path+ f'/Drammaturgia{numero_lezione}-{i+1}.m4a',output_path,30)

## 3.2 audio-to-text using Whisper Model

In [ ]:
model_size = "large-v2"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="auto", compute_type="int8")
#for i in range(numero_lezione):

#original = f"/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_segments/Drammaturgia{numero_lezione}-2_{y+1}.mp3"
#original2 = f'/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_transcripts/Drammaturgia{numero_lezione}-2_{y+1}.txt'

n_audio = 1

for y in range(1):
    time_stamp2 = time.time()
    segments, info = model.transcribe(f"/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_segments/Drammaturgia{numero_lezione}-{n_audio}_{y+1}.mp3", beam_size=5, language="it")
    with open(f'/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_transcripts/Drammaturgia{numero_lezione}-{n_audio}_{y+1}.txt', 'w') as f:
      for segment in segments:
          # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
          f.write(segment.text)
    time_stamp3 = time.time()
    print("sec: :", time_stamp3-time_stamp2, "s")

sec: : 180.57182335853577 s


In [ ]:
y = 4
time_stamp2 = time.time()
segments, info = model.transcribe(f"/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_segments/Drammaturgia{numero_lezione}-{n_audio}_{y+1}.mp3", beam_size=5, language="it")
with open(f'/content/drive/MyDrive/EMB_AudioSegments/Lecture_{numero_lezione}/audio_transcripts/Drammaturgia{numero_lezione}-{n_audio}_{y+1}.txt', 'w') as f:
  for segment in segments:
      # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
      f.write(segment.text)
time_stamp3 = time.time()
print("sec: :", time_stamp3-time_stamp2, "s")

sec: : 209.78843021392822 s


In [ ]:
def list_all_models():
    model_list = openai.Model.list()['data']
    model_ids = [x['id'] for x in model_list]
    model_ids.sort()
    print(model_ids)
list_all_models()
time_stamp2 = time.time()

with open('/content/drive/MyDrive/EMB_AudioSegments/Final/test_text.txt', 'r') as file:
    data_file = file.read()


response = openai.ChatCompletion.create(
    model = "gpt-4-0314",
    messages = [
        {"role": "user", "content": data_file + " This is the transcript of a lecture. Some words of the transcripts may be wrong so correct it when you write down the summary. Please give me a summary of this test such that I can used for study later during the exam session."},
    ]
)
time_stamp3 = time.time()
print("sec:", time_stamp3-time_stamp2, "s")
print(response['choices'][0]["message"]["content"])
print("total_tokens:", response['usage']["total_tokens"])
print("start writing")
with open('/content/drive/MyDrive/EMB_AudioSegments/Final/SummaryTest.txt', 'w') as f:
    f.write(response['choices'][0]["message"]["content"])
print("stop writing")
